In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import pyrsm as rsm 
import re

In [2]:
all_data = pd.read_csv("../Data/Final Dataset with Sentiment/All_Data_Vader.csv")

all_data.head()

/var/folders/79/jg9qrvxj5v1_ft3s9qfkpbl40000gn/T/ipykernel_79453/1421278644.py:1: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv("../Data/Final Dataset with Sentiment/All_Data_Vader.csv")


,Author,Timestamp,Comment,Article,Game Title,Console,Commenter Name,Reviewer Type,Date Commented,Score Number,Release Date,Description,Game Link,Article Link,Platform,Title,Upvotes,Vader_Score
0,shaftedman,1 day ago,"IGN Plus... just out of curiosity, does anyone...",daily-deals-the-last-of-us-part-i-spider-man-m...,GameSwitch Sports,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive
1,gamingdevil800,"3 July, 2021",This would be a lot more simplistic if you jus...,video-game-release-dates-ps4-ps5-xbox-one-seri...,GameFinal Fantasy 7 Rebirth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive
2,Snagglepuss000,"1 June, 2021",Back 4 Blood is NOT releasing in June. Pushed ...,video-game-release-dates-ps4-ps5-xbox-one-seri...,GameFinal Fantasy 7 Rebirth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neutral
3,seebs,"22 July, 2021","MS Flight Simulator, The Ascent, Hades, 12 Min...",video-game-release-dates-ps4-ps5-xbox-one-seri...,GameFinal Fantasy 7 Rebirth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Neutral
4,LedZepp9,"1 June, 2021","All the comments on ""there isn't any games, no...",video-game-release-dates-ps4-ps5-xbox-one-seri...,GameFinal Fantasy 7 Rebirth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive


In [3]:
all_data.shape, all_data.dtypes

((231306, 18),
 Author             object
 Timestamp          object
 Comment            object
 Article            object
 Game Title         object
 Console            object
 Commenter Name     object
 Reviewer Type      object
 Date Commented     object
 Score Number       object
 Release Date       object
 Description        object
 Game Link          object
 Article Link       object
 Platform           object
 Title              object
 Upvotes           float64
 Vader_Score        object
 dtype: object)

**Note:** `I had to clean up all the dates and makes the dtype date uniform so we can play with 'over time' trends.`

In [4]:
current_time = datetime(2024, 1, 6) # i did this around 1st week of Jan is what I remember 

In [5]:
def timestamp_fixer_upper(relative_date_str, current_time):
    if pd.isna(relative_date_str) or not isinstance(relative_date_str, str):
        return None
    
    parts = relative_date_str.split()
    if len(parts) >= 3:
        pass

    number, unit = parts[0], parts[1]
    try:
        number = int(number)
    except ValueError:
        return None

    if 'day' in unit:
        return current_time - timedelta(days=number)
    elif 'minute' in unit or 'min' in unit:
        return current_time - timedelta(minutes=number)
    elif 'hour' in unit or 'hr' in unit:
        return current_time - timedelta(hours=number)
    elif 'week' in unit or 'wk' in unit:
        return current_time - timedelta(weeks=number)
    elif 'month' in unit or 'mo' in unit:
        return current_time - timedelta(days=30 * number)
    elif 'year' in unit or 'yr' in unit:
        return current_time - timedelta(days=365 * number)
    else:
        return None

In [6]:
def derive_correct_timestamp(row, current_time): 
    timestamp = row['Timestamp']
    if not pd.isna(timestamp):
        custom_date_match = re.search(r"(?P<month>\w+) (?P<day>\d{1,2}), (?P<year>\d{4}) at (?P<hour>\d{1,2}):(?P<minute>\d{2}) (?P<ampm>am|pm)", timestamp)
        if custom_date_match:
            try:
                month = custom_date_match.group('month')
                day = int(custom_date_match.group('day'))
                year = int(custom_date_match.group('year'))
                hour = int(custom_date_match.group('hour'))
                minute = int(custom_date_match.group('minute'))
                ampm = custom_date_match.group('ampm')
                hour = hour if ampm == 'am' else (hour + 12 if hour < 12 else hour)
                
                return pd.to_datetime(f"{year}-{month}-{day} {hour}:{minute}", format='%Y-%B-%d %H:%M')
            except ValueError:
                pass
        elif 'ago' in timestamp:
            return timestamp_fixer_upper(timestamp, current_time)     
        else:
            try:
                return pd.to_datetime(timestamp)
            except:
                pass

    if pd.notna(row['Date Commented']):
        try:
            return pd.to_datetime(row['Date Commented'])
        except:
            pass

    if pd.notna(row['Release Date']):
        try:
            return pd.to_datetime(row['Release Date'])
        except:
            pass

    return None

In [7]:
all_data['Correct_TimeStamp'] = all_data.apply(lambda row: derive_correct_timestamp(row, current_time), axis=1)

In [8]:
print(all_data['Correct_TimeStamp'].isnull().sum(), all_data['Correct_TimeStamp'].notnull().sum(), len(all_data))

962 230344 231306


In [9]:
temp_df = all_data[all_data['Correct_TimeStamp'].isnull()]

In [10]:
temp_df["Date Commented"].isnull().sum(), temp_df["Release Date"].isnull().sum(), temp_df["Timestamp"].isnull().sum(), all_data["Correct_TimeStamp"].isnull().sum()

(962, 962, 962, 962)

In [11]:
temp_df["Timestamp"].unique()

array([nan], dtype=object)

In [12]:
print(temp_df[temp_df["Timestamp"].notnull()])

Empty DataFrame
Columns: [Author, Timestamp, Comment, Article, Game Title, Console, Commenter Name, Reviewer Type, Date Commented, Score Number, Release Date, Description, Game Link, Article Link, Platform, Title, Upvotes, Vader_Score, Correct_TimeStamp]
Index: []


In [13]:
print(all_data['Correct_TimeStamp'].isnull().sum(), all_data['Correct_TimeStamp'].notnull().sum(), len(all_data))

962 230344 231306


### this is the 'make the data up' 😅 section 
`im filling them in with the nearest rows which should help somewhat. `

In [14]:
all_data['Correct_TimeStamp'].fillna(method='ffill', inplace=True)

/var/folders/79/jg9qrvxj5v1_ft3s9qfkpbl40000gn/T/ipykernel_79453/892409252.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  all_data['Correct_TimeStamp'].fillna(method='ffill', inplace=True)


In [15]:
print(all_data['Correct_TimeStamp'].isnull().sum(), all_data['Correct_TimeStamp'].notnull().sum(), len(all_data))

0 231306 231306


---

In [16]:
all_data.shape, all_data.dtypes, all_data.columns

((231306, 19),
 Author                       object
 Timestamp                    object
 Comment                      object
 Article                      object
 Game Title                   object
 Console                      object
 Commenter Name               object
 Reviewer Type                object
 Date Commented               object
 Score Number                 object
 Release Date                 object
 Description                  object
 Game Link                    object
 Article Link                 object
 Platform                     object
 Title                        object
 Upvotes                     float64
 Vader_Score                  object
 Correct_TimeStamp    datetime64[ns]
 dtype: object,
 Index(['Author', 'Timestamp', 'Comment', 'Article', 'Game Title', 'Console',
        'Commenter Name', 'Reviewer Type', 'Date Commented', 'Score Number',
        'Release Date', 'Description', 'Game Link', 'Article Link', 'Platform',
        'Title', 'Upvotes', 'Va

In [17]:
# all_data.to_csv("Final_Vader_Time.csv", index=False)